In [12]:
import pandas as pd
import joblib
from fydjob.Database import Database
import fydjob.utils as utils
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
from fydjob.NLPFrame import NLPFrame

ndf = NLPFrame()

Loaded from /home/vlud/code/OWNER_vlad/find-your-dream-job/fydjob/output/nlp_frame.joblib
Token columns already found.
Processed column already found.
Loaded from /home/vlud/code/OWNER_vlad/find-your-dream-job/fydjob/output/nlp_frame.joblib


In [3]:
df = ndf.df[:20]

In [6]:
df.job_text_tokenized

0     [you, are, responsible, for, improvement, of, ...
1     [we, are, looking, for, a, senior, software, e...
2     [as, a, member, of, the, data, engineering, te...
3     [we, are, digittyio, an, international, startu...
4     [we, activate, data, for, our, clients, by, us...
5     [td, reply, is, an, innovation, and, marketing...
6     [the, spirit, of, a, startup, at, one, of, ger...
7     [responsibilities, as, senior, data, engineer,...
8     [freelancer, projekt, big, data, engineer, pyt...
9     [who, we, are, looking, for, we, are, looking,...
10    [junior, data, engineer, fmd, all, interviews,...
11    [checkout, charlie, is, an, international, pre...
12    [about, the, job, as, a, senior, data, enginee...
13    [your, position, as, a, data, engineer, mfd, y...
14    [overview, this, is, a, fulltime, position, ba...
15    [you, see, data, as, the, bloodstream, of, mod...
16    [about, the, job, as, a, senior, data, enginee...
17    [do, data, and, tech, get, you, pumped, do

In [8]:
df['clean'] = df.job_text_tokenized.apply(utils.lemmatize_words).apply(utils.remove_stopwords)
df['clean']

<ipython-input-8-bedc9e3e5dea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean'] = df.job_text_tokenized.apply(utils.lemmatize_words).apply(utils.remove_stopwords)


0     [responsible, improvement, taledos, search, ma...
1     [looking, senior, software, engineer, extensiv...
2     [member, data, engineering, team, responsible,...
3     [digittyio, international, startup, headquarte...
4     [activate, data, client, using, stateoftheart,...
5     [td, reply, innovation, marketing, consultancy...
6     [spirit, startup, one, germany, largest, famil...
7     [responsibility, senior, data, engineer, mfx, ...
8     [freelancer, projekt, big, data, engineer, pyt...
9     [looking, looking, solutionoriented, senior, d...
10    [junior, data, engineer, fmd, interview, done,...
11    [checkout, charlie, international, premium, pu...
12    [job, senior, data, engineer, pipeline, orches...
13    [position, data, engineer, mfd, help, shape, s...
14    [overview, fulltime, position, based, berlin, ...
15    [see, data, bloodstream, modern, technology, n...
16    [job, senior, data, engineer, pipeline, orches...
17    [data, tech, get, pumped, see, data, essen

In [13]:
texts = df['clean']
texts_tagged = [TaggedDocument(text, tags=['tag_'+str(tag)]) for tag, text in enumerate(texts)]

In [15]:
len(texts_tagged)

20

In [16]:
# build vocabulary with CBOW (dm=0) - instanciate model
model_dbow = Doc2Vec(documents=texts_tagged,
                     dm=0,
                     alpha=0.025,
                     vector_size=len(texts_tagged), 
                     min_count=1)

In [18]:
# train the model
for epoch in range(10):
    if epoch % 2 == 0:
        print(f'training epoch {epoch}')
    model_dbow.train(texts_tagged, total_examples=model_dbow.corpus_count, epochs=10)

training epoch 0
training epoch 2
training epoch 4
training epoch 6
training epoch 8


In [25]:
model_dbow

In [23]:
def find_similar_jobs(tokenized_job):
    ''' input: tokenized job offers 
        returns tags of top 5 most similar job offers and similarity probabilities
    '''
    
    # infer vector from text
    infer_vector = model_dbow.infer_vector(tokenized_job)
    # finds similar texts
    similar_documents = model_dbow.docvecs.most_similar([infer_vector], topn = 30)
    
    return similar_documents

In [24]:
find_similar_jobs(texts[0])

[('tag_0', 0.9975348114967346),
 ('tag_18', 0.3644728660583496),
 ('tag_11', 0.35717326402664185),
 ('tag_13', 0.3562268316745758),
 ('tag_15', 0.3551161289215088),
 ('tag_9', 0.35347431898117065),
 ('tag_7', 0.34985795617103577),
 ('tag_16', 0.34821438789367676),
 ('tag_8', 0.3468230366706848),
 ('tag_5', 0.34655648469924927),
 ('tag_6', 0.34473031759262085),
 ('tag_2', 0.34233200550079346),
 ('tag_12', 0.34160661697387695),
 ('tag_19', 0.3400558531284332),
 ('tag_3', 0.333818256855011),
 ('tag_1', 0.3330245018005371),
 ('tag_14', 0.32319244742393494),
 ('tag_10', 0.31951624155044556),
 ('tag_4', 0.31348657608032227),
 ('tag_17', 0.3073556125164032)]

In [4]:
df.head()

,job_id,job_title,job_text,company,location,job_info,job_link,query_text,source,tag_language,reviews
0,1,Data Scientist / Matching Engineer (m/w/d),You are responsible for improvement of Taledo’...,Taledo,Berlin,Data Scientist / Matching Engineer (m/w/d)\nTa...,data engineer,scrape_json,None,en,None
1,2,Senior Software Engineer - Data Platform,We are looking for a Senior Software Engineer ...,Zalando SE,Berlin,Senior Software Engineer - Data Platform\nZala...,data engineer,scrape_json,None,en,None
2,3,Senior Data Engineer (m/w/t),"As a member of the Data Engineering Team, you ...",Quandoo GmbH,Berlin,Senior Data Engineer (m/w/t)\nQuandoo GmbH17 B...,data engineer,scrape_json,None,en,None
3,4,Data Engineer (w/m/d),We are digitty.io – an international start-up ...,digitty.io,Berlin,Data Engineer (w/m/d)\ndigitty.io - Berlin,data engineer,scrape_json,None,en,None
4,5,Data Engineer,We activate data for our clients by using stat...,Gemma Analytics,Berlin,Data Engineer\nGemma Analytics - Berlin,data engineer,scrape_json,None,en,None


In [7]:
df.query_text.value_counts()

kaggle_1       5297
scrape_json     625
Name: query_text, dtype: int64

In [5]:
df.shape

(5922, 11)

# Trying a simpler approach. 

In [3]:
from fydjob.NLPFrame import NLPFrame

ndf = NLPFrame()

KeyboardInterrupt: 

In [2]:
ndf.head()

NameError: name 'ndf' is not defined